<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


In [29]:
import gradio as gr
import pandas as pd

cache = []  # List to store table rows
files = {}  # Dictionary to store files associated with each row
users = {"admin": "admin123", "user": "user123"}
current_user = None


def login(username, password):
    global current_user
    if username in users and users[username] == password:
        current_user = username
        return f"Welcome, {username}!", gr.update(visible=True)
    return "Invalid username or password", gr.update(visible=False)


def add_entry(order, project_id, project_name, amount, used, file):
    if current_user != "admin":
        return "Permission denied"
    remaining = float(amount) - float(used)

    # Attach the file if available
    file_name = "No file attached"
    if file is not None:
        files[order] = file.name
        file_name = f"Attached: {file.name}"

    cache.append([order, project_id, project_name, amount, used, remaining, file_name])
    df = pd.DataFrame(cache, columns=["ลำดับ", "รหัสโครงการ", "ชื่อโครงการ", "จำนวนเงิน", "ใช้แล้ว", "คงเหลือ", "ไฟล์แนบ"])
    return df


def get_table():
    df = pd.DataFrame(cache, columns=["ลำดับ", "รหัสโครงการ", "ชื่อโครงการ", "จำนวนเงิน", "ใช้แล้ว", "คงเหลือ", "ไฟล์แนบ"])
    return df


def download_file(order):
    if order in files:
        file_path = files[order]
        return file_path  # Gradio will allow downloading the file
    return "No file attached"


with gr.Blocks() as demo:
    with gr.Tab("Log In"):
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## Welcome Workers🏥!", elem_id="welcome-header")
            with gr.Column(scale=2):
                gr.Markdown("### Log In", elem_id="login-header")
                username = gr.Textbox(label="Username", placeholder="username", elem_id="username-box")
                password = gr.Textbox(label="Password", type="password", placeholder="********", elem_id="password-box")
                login_button = gr.Button("Log In", elem_id="login-button")
                login_output = gr.Textbox(label="Status", interactive=False, elem_id="login-status", visible=False)
                login_button.click(login, inputs=[username, password], outputs=[login_output, gr.State(False)])

    with gr.Tab("ทะเบียนคุม", visible=False) as tab_register:
        gr.Markdown("## ทะเบียนคุม")
        table_output = gr.Dataframe(headers=["ลำดับ", "รหัสโครงการ", "ชื่อโครงการ", "จำนวนเงิน", "ใช้แล้ว", "คงเหลือ", "ไฟล์แนบ"], interactive=False)
        admin_controls = gr.Column(visible=False)
        with admin_controls:
            with gr.Row():
                order = gr.Textbox(label="ลำดับ")
                project_id = gr.Textbox(label="รหัสโครงการ")
                project_name = gr.Textbox(label="ชื่อโครงการ")
                amount = gr.Textbox(label="จำนวนเงิน")
                used = gr.Textbox(label="ใช้แล้ว")
                file_input = gr.File(label="Select File")
            add_button = gr.Button("เพิ่มข้อมูล")
            add_button.click(add_entry, inputs=[order, project_id, project_name, amount, used, file_input], outputs=table_output)

        # For User tab (only for users to see download)
        user_controls = gr.Column(visible=False)
        with user_controls:
            download_order = gr.Textbox(label="Download from ลำดับ")
            download_button = gr.Button("Download File")
            download_output = gr.File(label="Download", interactive=False)
            download_button.click(download_file, inputs=[download_order], outputs=download_output)

    def show_tab(username, password):
        if username in users and users[username] == password:
            if username == "admin":
                return f"Welcome, {username}!", gr.update(visible=True), gr.update(visible=True), get_table(), gr.update(visible=False)
            return f"Welcome, {username}!", gr.update(visible=True), gr.update(visible=False), get_table(), gr.update(visible=True)
        return "Invalid username or password", gr.update(visible=False), gr.update(visible=False), None, gr.update(visible=False)

    login_button.click(show_tab, inputs=[username, password], outputs=[login_output, tab_register, admin_controls, table_output, user_controls])

    demo.css = """
    #welcome-header {
        font-size: 2rem;
        font-weight: bold;
        color: #1e88e5;
    }
    #login-header {
        font-size: 1.5rem;
        font-weight: bold;
    }
    #username-box, #password-box {
        margin-bottom: 1rem;
    }
    #login-button {
        background-color: #000000;
        color: white;
        font-weight: bold;
        border-radius: 5px;
    }
    """

demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://538429ff58a2c17c0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://538429ff58a2c17c0e.gradio.live


In [ ]:
# The correct credentials for admin are:
# Username: admin
# Password: admin123

# The correct credentials for user are:
# Username: user
# Password: user123